In [100]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torch.nn import Sequential,Conv1d,Linear,MaxPool1d,ReLU,LogSoftmax
from torch import flatten
import torchmetrics
from datasets import load_dataset
import os
import numpy as np
from tqdm.notebook import tqdm

class LightningMNISTClassifier(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.l0 = 1014
    self.num_classes = 14
    self.crepe_cnn = Sequential(
      Conv1d(in_channels = 70,out_channels = 256,kernel_size=7),
      ReLU(inplace=True),
      MaxPool1d(kernel_size=3), 
      Conv1d(in_channels = 256,out_channels = 256,kernel_size=7),
      ReLU(inplace=True),
      MaxPool1d(kernel_size=3),
      Conv1d(in_channels = 256,out_channels = 256,kernel_size=3),
      ReLU(inplace=True),
      Conv1d(in_channels = 256,out_channels = 256,kernel_size=3),
      ReLU(inplace=True),
      Conv1d(in_channels = 256,out_channels = 256,kernel_size=3),
      ReLU(inplace=True),
      Conv1d(in_channels = 256,out_channels = 256,kernel_size=3),
      ReLU(inplace=True),
      MaxPool1d(kernel_size=3),
    )
    self.crepe_dense = Sequential(
      Linear(int((self.l0 - 96)/27)*256,1024),
      ReLU(inplace=True),
      Linear(1024,1024), 
      ReLU(inplace=True),
      Linear(1024,self.num_classes),
      LogSoftmax(dim= 1)
    )
    self.train_acc = torchmetrics.Accuracy()
    self.train_prec = torchmetrics.Precision()
    self.train_rec = torchmetrics.Recall()
    self.train_f1 = torchmetrics.F1()
    
    self.val_acc = torchmetrics.Accuracy()
    self.val_prec = torchmetrics.Precision()
    self.val_rec = torchmetrics.Recall()
    self.val_f1 = torchmetrics.F1()
    
    self.test_acc = torchmetrics.Accuracy()
    self.test_prec = torchmetrics.Precision()
    self.test_rec = torchmetrics.Recall()
    self.test_f1 = torchmetrics.F1()
    
    self.test_preds = []
    self.test_labels = []
  

  def forward(self, x):
      batch_size, width, height = x.size()
      y = self.crepe_cnn(x)
      y = torch.flatten(y,start_dim=1)
      z = self.crepe_dense(y)
      return z

  def cross_entropy_loss(self, logits, labels):
    return F.nll_loss(logits, labels)

  def training_step(self, train_batch, batch_idx):
      x, y = train_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('train_loss', loss)
      self.train_acc(logits, y)
      self.log('train_acc', self.train_acc, on_step=False, on_epoch=True)
      self.train_prec(logits, y)
      self.log('train_prec', self.train_acc, on_step=False, on_epoch=True)
      self.train_rec(logits, y)
      self.log('train_rec', self.train_acc, on_step=False, on_epoch=True)
      self.train_f1(logits, y)
      self.log('train_f1', self.train_acc, on_step=False, on_epoch=True)
      return loss

  def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('val_loss', loss)
      self.val_acc(logits,y)
      self.log('val_acc', self.val_acc, on_step=False, on_epoch=True)
      self.val_prec(logits,y)
      self.log('val_prec', self.val_acc, on_step=False, on_epoch=True)
      self.val_rec(logits,y)
      self.log('val_rec', self.val_acc, on_step=False, on_epoch=True)
      self.val_f1(logits,y)
      self.log('val_f1', self.val_acc, on_step=False, on_epoch=True)

  def test_step(self, test_batch, batch_idx):
      x, y = test_batch
      logits = self.forward(x)
      self.test_labels += y.cpu().numpy().tolist()
      self.test_preds += np.argmax(logits.cpu().numpy(), axis=1).flatten().tolist()
      loss = self.cross_entropy_loss(logits, y)
      self.log('test_loss', loss)
      self.test_acc(logits,y)
      self.log('test_acc', self.test_acc)
      self.test_prec(logits,y)
      self.log('test_prec', self.test_acc)
      self.test_rec(logits,y)
      self.log('test_rec', self.test_acc)
      self.test_f1(logits,y)
      self.log('test_f1', self.test_acc)
      
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
def get_metrics(y_true, y_pred):
    f = open(f'report_crepe.txt','w+')
    result1 = classification_report(y_true, y_pred)
    print('Classification Report: ', result1)
    f.write('\nClassification Report: \n')
    f.write(result1)
    # print(type(result1))
    # df = pd.DataFrame(result1).transpose()
    # df.to_csv('report.csv')
    result2 = accuracy_score(y_true, y_pred)
    print('Accuracy: ', result2, "\n\n")
    f.write('\nAccuracy Report: \n')
    f.write(str(result2))

In [101]:
class MNISTDataModule(pl.LightningDataModule):
    def collate_fn(self,data):
        from torch.nn.functional import one_hot
        examples, labels = zip(*data)
        labels = torch.tensor(labels)
        batch_size = len(examples)
        length = len(examples[0])
        vocab_size = 70
        features = torch.zeros((batch_size, length,vocab_size))
        
        for ind,example in enumerate(examples):
            features[ind,:,:] = one_hot(example.to(torch.int64))

        return torch.transpose(features,dim0=2,dim1=1).float(), labels.long()
    
    def setup(self, stage):    
        from sklearn.preprocessing import label_binarize
        from torch.utils.data import TensorDataset, random_split
        from sklearn.preprocessing import LabelEncoder
        from collections import defaultdict
        dataset = load_dataset("dbpedia_14")
        train = dataset['train']
        train = train.shuffle(seed=42)[:]
        test = dataset['test']
        test = test.shuffle(seed=42)[:]
        length = 1014
        labels = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\|_@#$%^&*~`+=<>()[]{}\n"
        label_dict = defaultdict(lambda:69)
        for ind,i in enumerate(labels):
            label_dict[i] = ind 
        train['content'] = [list(i.lower())[:length] + [' '] * (length - len(list(i.lower())[:length])) for i in tqdm(train['content'])]
        train['content'] = [[label_dict[j] for j in i]for i in tqdm(train['content'])]
        test['content'] = [list(i.lower())[:length] + [' '] * (length - len(list(i.lower())[:length])) for i in tqdm(test['content'])]
        test['content'] = [[label_dict[j] for j in i]for i in tqdm(test['content'])]
        train = TensorDataset(torch.Tensor(train['content']),torch.tensor(train['label']))
        self.mnist_test = TensorDataset(torch.Tensor(test['content']),torch.tensor(test['label']))
        self.mnist_train,self.mnist_val = random_split(train,[int(0.85*len(train)),len(train)-int(0.85*len(train))])
        
        
    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=64,collate_fn=self.collate_fn)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=64,collate_fn=self.collate_fn)
    
    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=64,collate_fn=self.collate_fn)
    

In [102]:
data_module = MNISTDataModule()

# train
model = LightningMNISTClassifier()
trainer = pl.Trainer(gpus= 1,max_epochs= 10,log_every_n_steps=10)

trainer.fit(model, data_module)
trainer.test(datamodule= data_module,model=model)
get_metrics(model.test_labels,model.test_preds)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Reusing dataset d_bpedia14 (C:\Users\KAWSHIK\.cache\huggingface\datasets\d_bpedia14\dbpedia_14\2.0.0\7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e)
100%|██████████| 2/2 [00:00<00:00, 80.22it/s]
Loading cached shuffled indices for dataset at C:\Users\KAWSHIK\.cache\huggingface\datasets\d_bpedia14\dbpedia_14\2.0.0\7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e\cache-4f9bd4edcc24be65.arrow
Loading cached shuffled indices for dataset at C:\Users\KAWSHIK\.cache\huggingface\datasets\d_bpedia14\dbpedia_14\2.0.0\7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e\cache-d75e0d932d1c3620.arrow


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

...

d:\IREL\Major Project\ire1\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
                                                                      

d:\IREL\Major Project\ire1\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/157 [00:00<?, ?it/s] 64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Epoch 0:   1%|          | 1/157 [00:00<00:28,  5.54it/s, loss=2.64, v_num=38]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Epoch 0:   1%|▏         | 2/157 [00:00<00:26,  5.83it/s, loss=2.65, v_num=38]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Epoch 0:   2%|▏         | 3/157 [00:00<00:27,  5.68it/s, loss=2.65, v_num=38]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Epoch 0:   3%|▎         | 4/157 [00:00<00:26,  5.82it/s, loss=2.65, v_num=38]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Epoch 0:   3%|▎         | 5/157 [00:00<00:26,  5.77it/s, loss=2.65, v_num=38]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch

MisconfigurationException: No `test_dataloader()` method defined to run `Trainer.test`.

Reusing dataset d_bpedia14 (C:\Users\KAWSHIK\.cache\huggingface\datasets\d_bpedia14\dbpedia_14\2.0.0\7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e)
100%|██████████| 2/2 [00:00<00:00, 64.63it/s]
Loading cached shuffled indices for dataset at C:\Users\KAWSHIK\.cache\huggingface\datasets\d_bpedia14\dbpedia_14\2.0.0\7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e\cache-4f9bd4edcc24be65.arrow
Loading cached shuffled indices for dataset at C:\Users\KAWSHIK\.cache\huggingface\datasets\d_bpedia14\dbpedia_14\2.0.0\7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e\cache-d75e0d932d1c3620.arrow


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

Restoring states from the checkpoint path at d:\Sem5\SMAI\lightning_logs\version_38\checkpoints\epoch=9-step=1329.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at d:\Sem5\SMAI\lightning_logs\version_38\checkpoints\epoch=9-step=1329.ckpt
d:\IREL\Major Project\ire1\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Testing:   1%|          | 1/157 [00:00<00:18,  8.36it/s]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Testing:   2%|▏         | 3/157 [00:00<00:13, 11.47it/s]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Testing:   3%|▎         | 5/157 [00:00<00:13, 10.89it/s]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final Output torch.Size([64, 14])
Testing:   4%|▍         | 7/157 [00:00<00:23,  6.40it/s]64 70 1014
torch.float32
Y dimension torch.Size([64, 256, 34])
Final O

[{'test_loss': 2.6409757137298584,
  'test_acc': 0.07240000367164612,
  'test_prec': 0.07240000367164612,
  'test_rec': 0.07240000367164612,
  'test_f1': 0.07240000367164612}]

In [ ]:
# tensorboard --logdir=lightning_logs/

In [ ]:
# def collate_fn(data):
#     """
#        data: is a list of tuples with (example, label, length)
#              where 'example' is a tensor of arbitrary shape
#              and label/length are scalars
#     """
#     print(data)
#     from torch.nn.functional import one_hot
#     examples, labels = zip(*data)
#     labels = torch.tensor(labels)
#     batch_size = len(examples)
#     length = len(examples[0])
#     vocab_size = 70
#     features = torch.zeros((batch_size, length,vocab_size))
    
#     for ind,example in enumerate(examples):
#         features[ind,:,:] = one_hot(example.to(torch.int64))
        
#     return features.float(), labels.long()

In [ ]:
# from sklearn.preprocessing import label_binarize
# from torch.utils.data import TensorDataset, random_split
# from sklearn.preprocessing import LabelEncoder
# from collections import defaultdict
# dataset = load_dataset("dbpedia_14")
# train = dataset['train']
# train = train.shuffle(seed=42)[:10]
# test = dataset['test']
# test = test.shuffle(seed=42)[:10]
# length = 1014
# labels = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\|_@#$%^&*~`+=<>()[]{}\n"
# label_dict = defaultdict(lambda:69)
# for ind,i in enumerate(labels):
#     label_dict[i] = ind 
# train['content'] = [list(i.lower())[:length] + [' '] * (length - len(list(i.lower())[:length])) for i in tqdm(train['content'])]
# train['content'] = [[ label_dict[j] for j in i]for i in tqdm(train['content'])]
# len(train['content']),len(train['content'][0])
# # test['content'] = [list(i.lower())[:length] + [' '] * (length - len(list(i.lower())[:length])) for i in tqdm(test['content'])]
# # train['content'] = []
# # label_encoder = LabelEncoder()
# # y = label_encoder.fit_transform()
# # train['content'] = [label_binarize(i,classes=labels) for i in tqdm(train['content'])]
# # train['content'] = torch.tensor(np.array(train['content']))
# # test['content'] = [label_binarize(i,classes=labels) for i in tqdm(test['content'])]
# # test['content'] = torch.tensor(np.array(train['content']))

# # print(torch.transpose(train['content'],dim0=2,dim1=1).float().shape)
# # print(torch.tensor(train['label']).shape)
# # print(torch.transpose(test['content'],dim0=2,dim1=1).float().shape)
# # print(torch.tensor(test['label']).shape)

# # torch.save(torch.transpose(train['content'],dim0=2,dim1=1).float(), 'traincontent.pt')
# # torch.save(torch.tensor(train['label']),'trainlabel.pt')
# # torch.save(torch.transpose(test['content'],dim0=2,dim1=1).float(),'testcontent.pt')
# # torch.save(torch.tensor(test['label']),'testlabel.pt')

# train = TensorDataset(torch.Tensor(train['content']),torch.tensor(train['label']))
# # train = TensorDataset(torch.transpose(train['content'],dim0=2,dim1=1).float(),torch.tensor(train['label']))
# # mnist_test = TensorDataset(torch.transpose(test['content'],dim0=2,dim1=1).float(),torch.tensor(test['label']))
# # mnist_train,mnist_val = random_split(train,[450,50])
# train_dat = DataLoader(train, batch_size=2,collate_fn=collate_fn)
# iter_dat = iter(train_dat)
# first =next(iter_dat)[0]
# print(first,len(first))
# print(first.shape)
# # cnn = Conv1d(in_channels = 69,out_channels = 256,kernel_size=7)
# # output = cnn(first)

In [ ]:
# a = [1,2,3,4,5]
# a[:10]

In [ ]:
# F.one_hot(torch.arange(0, 5) % 3)
# F.one_hot(torch.arange(0, 10), num_classes=5)
# F.one_hot(torch.arange(0, 6).view(3,2) % 3)

In [ ]:
# data = [(torch.randint(0,70,(1014,)), 0, 1014 ) for i in range(5)]
# print(data)
# from torch.nn.functional import one_hot
# examples, labels, lengths = zip(*data)
# batch_size = len(examples)
# length = len(examples[0])
# vocab_size = 70
# features = torch.zeros((batch_size, length,vocab_size))
# for ind,example in enumerate(examples):
#     features[ind,:,:] = one_hot(example)
# return features.float(), labels.long(), lengths.long()

In [ ]:
# from sklearn.preprocessing import label_binarize
        # from torch.utils.data import TensorDataset, random_split
        # dataset = load_dataset("dbpedia_14")
        # train = dataset['train']
        # train = train.shuffle(seed=42)[:10000]
        # test = dataset['test']
        # test = test.shuffle(seed=42)[:10000]
        # length = 1014
        # train['content'] = [list(i.lower())[:length] + [' '] * (length - len(list(i.lower())[:length])) for i in tqdm(train['content'])]
        # test['content'] = [list(i.lower())[:length] + [' '] * (length - len(list(i)[:length])) for i in tqdm(test['content'])]
        # for i in train['content']:
        #     if len(i) > length:
        #         print(len(i))
        #         print(i)
        # labels = list("abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\|_@#$%^&*~`+=<>()[]{}\n")
        # train['content'] = [label_binarize(i,classes=labels) for i in tqdm(train['content'])]
        # print("Done character one hotting")
        # train['content'] = torch.tensor(np.array(train['content']))
        # test['content'] = [label_binarize(i,classes=labels) for i in tqdm(test['content'])]
        # test['content'] = torch.tensor(np.array(train['content']))
        # train = TensorDataset(torch.transpose(train['content'],dim0=2,dim1=1).float(),torch.tensor(train['label']))
        # self.mnist_test = TensorDataset(torch.transpose(test['content'],dim0=2,dim1=1).float(),torch.tensor(test['label']))
        # self.mnist_train,self.mnist_val = random_split(train,[int(0.85*len(train)),len(train)-int(0.85*len(train))])